In [1]:
cd '/content/drive/My Drive/ML/Final'

/content/drive/My Drive/ML/Final


In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from preprocessing.imagetoarraypreprocessor import ImageToArrayPreprocessor
from preprocessing.simplepreprocessor import SimplePreprocessor

from dataloader.simpledatasetloader import SimpleDatasetLoader

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model

from transfer.fcheadnet import FCHeadNet

import argparse
from imutils import paths
import numpy as np
import os

In [3]:
path_data = 'Data/Train_data'

In [4]:
imagePaths = list(paths.list_images(path_data))

In [5]:
len(imagePaths)

13729

In [6]:
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

In [7]:
# initialize the image preprocessors
sp = SimplePreprocessor(224,224)
iap = ImageToArrayPreprocessor()

In [ ]:
sdl = SimpleDatasetLoader(preprocessors=[sp, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

[INFO] processed 500/13729
[INFO] processed 1000/13729
[INFO] processed 1500/13729
[INFO] processed 2000/13729
[INFO] processed 2500/13729
[INFO] processed 3000/13729
[INFO] processed 3500/13729
[INFO] processed 4000/13729
[INFO] processed 4500/13729
[INFO] processed 5000/13729
[INFO] processed 5500/13729
[INFO] processed 6000/13729
[INFO] processed 6500/13729
[INFO] processed 7000/13729
[INFO] processed 7500/13729
[INFO] processed 8000/13729
[INFO] processed 8500/13729
[INFO] processed 9000/13729
[INFO] processed 9500/13729
[INFO] processed 10000/13729
[INFO] processed 10500/13729
[INFO] processed 11000/13729
[INFO] processed 11500/13729
[INFO] processed 12000/13729
[INFO] processed 12500/13729
[INFO] processed 13000/13729
[INFO] processed 13500/13729


In [1]:
data.shape

NameError: ignored

In [ ]:
#split training: 80%, testing: 20%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, random_state=42)

NameError: ignored

In [ ]:
data_1     = trainX[0:4500]
labels_1   = trainY[0:4500]

data_2     = data[4500:9000]
labels_2   = labels[4500:9000]

data_3     = data[9000:13729]
labels_3   = labels[9000:13729]

In [ ]:
# convert the labels from integers to vectors
labels_1 = LabelBinarizer().fit_transform(labels_1)
labels_2 = LabelBinarizer().fit_transform(labels_2)
labels_3 = LabelBinarizer().fit_transform(labels_3)

In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
horizontal_flip=True, fill_mode="nearest")

In [ ]:
baseModel = VGG16(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
# initialize the new head of the network, a set of FC layers
# followed by a softmax classifier
headModel = FCHeadNet.build(baseModel, len(classNames), 256)

In [ ]:
# place the head FC model on top of the base model,
# become the actual model
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
# loop over all layers in the base model and freeze them so they
# will not be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])

In [ ]:
# train the head of the network for a few epochs (all other
# layers are frozen) 
print("[INFO] training head...")
model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
                    validation_data=(testX, testY), epochs=20,
                    steps_per_epoch=len(trainX) // 32, verbose=1)